In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Carregar os dados
gruposDFnomeDasColunas = [
    "nome",
    "classificacao",
    "frequencia_feminina",
    "frequencia_masculina",
    "frequencial_total",
    "proporcao",
    "nomes_alternativos"
]
gruposDF = pd.read_csv('grupos.csv', names=gruposDFnomeDasColunas, header=0)

nomesDSnomeDasColunas = [
    "nomes_alternativos",
    "classificacao",
    "primeiro_nome",
    "frequencia_feminina",
    "frequencia_masculina",
    "frequencia_total",
    "frequencia_grupo",
    "nome_grupo",
    "proporcao"
]
nomesDF = pd.read_csv("nomes.csv", names=nomesDSnomeDasColunas, header=0)

# Renomear colunas para consistência
gruposDF = gruposDF[['nome', 'classificacao']].rename(columns={'nome': 'nome', 'classificacao': 'classificacao'})
nomesDF = nomesDF[['primeiro_nome', 'classificacao']].rename(columns={'primeiro_nome': 'nome', 'classificacao': 'classificacao'})


gruposDF.dropna(inplace=True)
nomesDF.dropna(inplace=True)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
gruposDF['classificacao'] = label_encoder.fit_transform(gruposDF['classificacao'])
nomesDF['classificacao'] = label_encoder.fit_transform(nomesDF['classificacao'])

# Unir os DataFrames
data = pd.concat([gruposDF, nomesDF])



In [36]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenização dos dados
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['nome'])
sequences = tokenizer.texts_to_sequences(data['nome'])
word_index = tokenizer.word_index

# Padronização das sequências
max_length = max([len(seq) for seq in sequences])
X = pad_sequences(sequences, maxlen=max_length, padding='post')
y = data['classificacao']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construção do modelo
model = Sequential()
model.add(Embedding(len(word_index) + 1, 128, input_length=max_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilação do modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinamento do modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Avaliação do modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Acurácia: {accuracy*100:.2f}%')

Epoch 1/10


2024-10-20 04:59:16.192859: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
2024-10-20 04:59:16.193179: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.


UnknownError: Graph execution error:

Detected at node sequential_2/dense_5/Sigmoid defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/augusto/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/augusto/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/home/augusto/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/augusto/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/augusto/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/augusto/anaconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/home/augusto/anaconda3/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/home/augusto/anaconda3/lib/python3.

In [38]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'